## Benchmarking Netlist Generator

In [ ]:
import sys

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")
from collections import Counter

import yaml

from PhotonicsAI.Photon import llm_api, utils

with open("../../Frontend/prompts.yaml") as file:
    prompts = yaml.safe_load(file)

db_docs = utils.search_directory_for_docstrings(directory="../../KnowledgeBase")
list_of_docs = [i["docstring"] for i in db_docs]
list_of_cnames = [i["class_name"] for i in db_docs]

d = {
    "list_of_docs": list_of_docs,
    "list_of_cnames": list_of_cnames,
}

llm_api_selection_ = [
    "fireworks: llama-v3-70b-instruct",
    # 'fireworks: llama-v3-8b-instruct',
    "openai: gpt-4o",
    # 'openai: gpt-3.5',
    #   'groq: llama3-70b',
    # 'groq: llama3-8b',
    # 'fireworks: qwen2-72b-instruct',
    # 'fireworks: mixtral-8x22b-instruct',
    "google: gemini-1.5-flash",
    #   'google: gemini-1.5-pro',
]

In [ ]:
N = 10

for llm_api_selection in llm_api_selection_:
    res_connections = []
    res_search = []
    d["llm_api_selection"] = llm_api_selection
    print("-------------")
    print(llm_api_selection)
    for _i in range(N):
        try:
            d["prompt"] = (
                "Five cascaded MZIs, each with two input and two output ports."
            )

            d["raw_component_list"] = llm_api.call_llm(
                d["prompt"], prompts["classify"], d["llm_api_selection"]
            )
            d["db_component_list"], d["modified_yaml_data_all"] = (
                llm_api.yaml_components_search(d)
            )

            _data = yaml.safe_load(d["db_component_list"])
            res_search.append(list(_data.values())[0]["component"])
        except:
            res_search.append("ERROR")

        try:
            r_netlist = llm_api.parsed_prompt_to_netlist(d)
            res_connections.append(yaml.safe_load(r_netlist)["connections"])
        except:
            res_connections.append({"ERROR": "ERROR"})

    # Count the search results
    counter = Counter(res_search)
    unique_items = list(counter.items())
    unique_items.sort(key=lambda x: x[0])
    print("Search results:")
    for item, count in unique_items:
        print(f"    {count}: {item}")

    # count the netlist connections
    dict_tuples = [tuple(sorted(d.items())) for d in res_connections]
    counter = Counter(dict_tuples)
    sorted_counts = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    print("Connections:")
    for dict_tuple, count in sorted_counts:
        unique_dict = dict(dict_tuple)
        print(f"    Count: {count}: {unique_dict}")